In [1]:
pip install xgboost lightgbm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 3.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.6 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier, LGBMRegressor 
from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier, CatBoostRegressor

# import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'March-Mania-2023/Evan-Data/MNCAA_train_538.csv'
file_key_2 = 'March-Mania-2023/Evan-Data/MNCAA_test_538.csv'
file_key_3 = 'March-Mania-2023/Evan-Data/WNCAA_train_538.csv'
file_key_4 = 'March-Mania-2023/Evan-Data/WNCAA_test_538.csv'
file_key_5 = 'March-Mania-2023/Evan-Data/SampleSubmission2023.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

bucket_object_5 = bucket.Object(file_key_5)
file_object_5 = bucket_object_5.get()
file_content_stream_5 = file_object_5.get('Body')

## Reading data files
man_train = pd.read_csv(file_content_stream_1)
man_test = pd.read_csv(file_content_stream_2)
woman_train = pd.read_csv(file_content_stream_3)
woman_test = pd.read_csv(file_content_stream_4)

submission = pd.read_csv(file_content_stream_5)

man_train['target'] = np.where(man_train['ResultDiff'] > 0, 1, 0)
man_train_2016 = man_train[man_train['Season'] >= 2016].reset_index(drop = True)

woman_train['target'] = np.where(woman_train['ResultDiff'] > 0, 1, 0)
woman_train_2016 = woman_train[woman_train['Season'] >= 2016].reset_index(drop = True)

/tmp/ipykernel_8954/1750915339.py:57: DtypeWarning: Columns (46,55) have mixed types. Specify dtype option on import or set low_memory=False.
  man_test = pd.read_csv(file_content_stream_2)
/tmp/ipykernel_8954/1750915339.py:59: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  woman_test = pd.read_csv(file_content_stream_4)


# Solution 1
## Phase 1: Man

In [3]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2']

X = man_train[to_select]
Y = man_train['ResultDiff']

man_test_tour =  man_test[~(man_test['Seed1'].isnull() | man_test['Seed2'].isnull())].reset_index(drop = True)
man_test_IDS = man_test_tour['ID'] 
man_test_tour = man_test_tour[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('man_XGB_Phase_1_42_Optuna_Hyperparameters.csv')
xgb_preds = list()

for i in tqdm(range(100)):

    xgb_md = XGBRegressor(tree_method = 'hist', 
                          max_depth = xgb_params['max_depth'][0],
                          learning_rate = xgb_params['learning_rate'][0],
                          n_estimators = xgb_params['n_estimators'][0],
                          gamma = xgb_params['gamma'][0],
                          min_child_weight = xgb_params['min_child_weight'][0],
                          colsample_bytree = xgb_params['colsample_bytree'][0],
                          subsample = xgb_params['subsample'][0],
                          random_state = 1).fit(X, Y)

    xgb_pred = xgb_md.predict(man_test_tour)
    xgb_preds.append(xgb_pred)
    
##############
## LightGBM ##
##############

lgb_params = pd.read_csv('man_LightGBM_Phase_1_42_Optuna_Hyperparameters.csv')
lgb_preds = list()

for i in tqdm(range(100)):

    lgb_md = LGBMRegressor(boosting_type = 'gbdt', 
                           n_estimators = lgb_params['n_estimators'][0],
                           learning_rate = lgb_params['learning_rate'][0],
                           max_depth = lgb_params['max_depth'][0],
                           lambda_l1 = lgb_params['lambda_l1'][0],
                           lambda_l2 = lgb_params['lambda_l2'][0],
                           num_leaves = lgb_params['num_leaves'][0],
                           bagging_fraction = lgb_params['bagging_fraction'][0],
                           feature_fraction = lgb_params['feature_fraction'][0], 
                           random_state = i).fit(X, Y)

    lgb_pred = lgb_md.predict(man_test_tour)
    lgb_preds.append(lgb_pred)
    
##################
## HistGradient ##
##################

hist_params = pd.read_csv('man_Hist_Phase_1_42_Optuna_Hyperparameters.csv')
hist_preds = list()

for i in tqdm(range(100)):

    hist_md = HistGradientBoostingRegressor(l2_regularization = hist_params['l2_regularization'][0],
                                            early_stopping = False,
                                            learning_rate = hist_params['learning_rate'][0],
                                            max_iter = hist_params['max_iter'][0],
                                            max_depth = hist_params['max_depth'][0],
                                            max_bins = hist_params['max_bins'][0],
                                            min_samples_leaf = hist_params['min_samples_leaf'][0],
                                            max_leaf_nodes = hist_params['max_leaf_nodes'][0],
                                            random_state = i).fit(X, Y)

    hist_pred = hist_md.predict(man_test_tour)
    hist_preds.append(hist_pred)

##############
## Ensemble ##
##############

xgb_pred = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
lgb_pred = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
hist_pred = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
man_test_tour['ResultDiff'] = np.round_(ens_pred, decimals = 0).astype(int)
man_test_tour['ID'] = man_test_IDS

########################
## Appending 538 data ##
########################

data_538 = ['ID',
            'X1_team_rating',
            'X1_rd1_win',
            'X1_rd2_win',
            'X1_rd3_win',
            'X1_rd4_win',
            'X1_rd5_win',
            'X1_rd6_win',
            'X1_rd7_win',
            'X2_team_rating',
            'X2_rd1_win',
            'X2_rd2_win',
            'X2_rd3_win',
            'X2_rd4_win',
            'X2_rd5_win',
            'X2_rd6_win',
            'X2_rd7_win']

man_test_538 = man_test[data_538]
man_test_tour = pd.merge(man_test_tour, man_test_538, on = 'ID', how = 'left')

  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=0.1713152100541493, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1713152100541493
[LightGBM] [Warning] bagging_fraction is set=0.8443456049471257, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8443456049471257
[LightGBM] [Warning] feature_fraction is set=0.2439888911439529, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2439888911439529
[LightGBM] [Warning] lambda_l2 is set=1.290608460937753, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.290608460937753


  1%|          | 1/100 [00:07<12:25,  7.53s/it]

[LightGBM] [Warning] lambda_l1 is set=0.1713152100541493, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1713152100541493
[LightGBM] [Warning] bagging_fraction is set=0.8443456049471257, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8443456049471257
[LightGBM] [Warning] feature_fraction is set=0.2439888911439529, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2439888911439529
[LightGBM] [Warning] lambda_l2 is set=1.290608460937753, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.290608460937753


  2%|▏         | 2/100 [00:14<12:07,  7.42s/it]

...

 98%|█████████▊| 98/100 [13:17<00:14,  7.49s/it]

[LightGBM] [Warning] lambda_l1 is set=0.1713152100541493, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1713152100541493
[LightGBM] [Warning] bagging_fraction is set=0.8443456049471257, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8443456049471257
[LightGBM] [Warning] feature_fraction is set=0.2439888911439529, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2439888911439529
[LightGBM] [Warning] lambda_l2 is set=1.290608460937753, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.290608460937753


 99%|█████████▉| 99/100 [13:25<00:07,  7.50s/it]

[LightGBM] [Warning] lambda_l1 is set=0.1713152100541493, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1713152100541493
[LightGBM] [Warning] bagging_fraction is set=0.8443456049471257, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8443456049471257
[LightGBM] [Warning] feature_fraction is set=0.2439888911439529, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2439888911439529
[LightGBM] [Warning] lambda_l2 is set=1.290608460937753, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.290608460937753


100%|██████████| 100/100 [21:18<00:00, 12.79s/it]


## Phase 2: Man

In [4]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2', 
             'ResultDiff',
             'X1_team_rating',
             'X1_rd1_win',
             'X1_rd2_win',
             'X1_rd3_win',
             'X1_rd4_win',
             'X1_rd5_win',
             'X1_rd6_win',
             'X1_rd7_win',
             'X2_team_rating',
             'X2_rd1_win',
             'X2_rd2_win',
             'X2_rd3_win',
             'X2_rd4_win',
             'X2_rd5_win',
             'X2_rd6_win',
             'X2_rd7_win']

X = man_train_2016[to_select]
Y = man_train_2016['target']

man_test_tour_ID = man_test_tour['ID']
man_test_tour = man_test_tour[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('man_XGB_Phase_2_42_Optuna_Hyperparameters.csv')
xgb_preds = list()

for i in tqdm(range(100)):

    xgb_md = XGBClassifier(tree_method = 'hist', 
                           max_depth = xgb_params['max_depth'][0],
                           learning_rate = xgb_params['learning_rate'][0],
                           n_estimators = xgb_params['n_estimators'][0],
                           gamma = xgb_params['gamma'][0],
                           min_child_weight = xgb_params['min_child_weight'][0],
                           colsample_bytree = xgb_params['colsample_bytree'][0],
                           subsample = xgb_params['subsample'][0],
                           random_state = i).fit(X, Y)

    xgb_pred = xgb_md.predict_proba(man_test_tour)[:, 1]
    xgb_preds.append(xgb_pred)

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('man_LightGBM_Phase_2_42_Optuna_Hyperparameters.csv')
lgb_preds = list()

for i in tqdm(range(100)):
    
    lgb_md = LGBMClassifier(boosting_type = 'gbdt', 
                            n_estimators = lgb_params['n_estimators'][0],
                            learning_rate = lgb_params['learning_rate'][0],
                            max_depth = lgb_params['max_depth'][0],
                            lambda_l1 = lgb_params['lambda_l1'][0],
                            lambda_l2 = lgb_params['lambda_l2'][0],
                            num_leaves = lgb_params['num_leaves'][0],
                            bagging_fraction = lgb_params['bagging_fraction'][0],
                            feature_fraction = lgb_params['feature_fraction'][0],
                            random_state = i).fit(X, Y)

    lgb_pred = lgb_md.predict_proba(man_test_tour)[:, 1]
    lgb_preds.append(lgb_pred)

##################
## HistGradient ##
##################

hist_params = pd.read_csv('man_Hist_Phase_2_42_Optuna_Hyperparameters.csv')
hist_preds = list()

for i in tqdm(range(100)):

    hist_md = HistGradientBoostingClassifier(l2_regularization = hist_params['l2_regularization'][0],
                                             early_stopping = False,
                                             learning_rate = hist_params['learning_rate'][0],
                                             max_iter = hist_params['max_iter'][0],
                                             max_depth = hist_params['max_depth'][0],
                                             max_bins = hist_params['max_bins'][0],
                                             min_samples_leaf = hist_params['min_samples_leaf'][0],
                                             max_leaf_nodes = hist_params['max_leaf_nodes'][0], 
                                             random_state = i).fit(X, Y)

    hist_pred = hist_md.predict_proba(man_test_tour)[:, 1]
    hist_preds.append(hist_pred)

##############
## Ensemble ##
##############

xgb_pred = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
lgb_pred = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
hist_pred = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
man_pred = pd.DataFrame({'ID': man_test_tour_ID, 'Pred': ens_pred})
man_pred.head()

  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=1.6568558853289863, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.6568558853289863
[LightGBM] [Warning] bagging_fraction is set=0.8504282163263281, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8504282163263281
[LightGBM] [Warning] feature_fraction is set=0.705433568927063, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.705433568927063
[LightGBM] [Warning] lambda_l2 is set=5.241653664300623, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.241653664300623


  1%|          | 1/100 [00:02<04:38,  2.81s/it]

[LightGBM] [Warning] lambda_l1 is set=1.6568558853289863, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.6568558853289863
[LightGBM] [Warning] bagging_fraction is set=0.8504282163263281, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8504282163263281
[LightGBM] [Warning] feature_fraction is set=0.705433568927063, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.705433568927063
[LightGBM] [Warning] lambda_l2 is set=5.241653664300623, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.241653664300623


  2%|▏         | 2/100 [00:11<10:26,  6.39s/it]

...

[LightGBM] [Warning] lambda_l1 is set=1.6568558853289863, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.6568558853289863
[LightGBM] [Warning] bagging_fraction is set=0.8504282163263281, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8504282163263281
[LightGBM] [Warning] feature_fraction is set=0.705433568927063, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.705433568927063
[LightGBM] [Warning] lambda_l2 is set=5.241653664300623, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.241653664300623


 99%|█████████▉| 99/100 [05:09<00:02,  2.78s/it]

[LightGBM] [Warning] lambda_l1 is set=1.6568558853289863, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.6568558853289863
[LightGBM] [Warning] bagging_fraction is set=0.8504282163263281, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8504282163263281
[LightGBM] [Warning] feature_fraction is set=0.705433568927063, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.705433568927063
[LightGBM] [Warning] lambda_l2 is set=5.241653664300623, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.241653664300623


100%|██████████| 100/100 [04:15<00:00,  2.56s/it]


,ID,Pred
0,2023_1104_1112,0.948612
1,2023_1104_1113,0.951004
2,2023_1104_1116,0.950284
3,2023_1104_1120,0.950387
4,2023_1104_1124,0.949926


## Phase 1: Woman

In [ ]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2']

X = woman_train[to_select]
Y = woman_train['ResultDiff']

woman_test_tour =  woman_test[~(woman_test['Seed1'].isnull() | woman_test['Seed2'].isnull())].reset_index(drop = True)
woman_test_IDS = woman_test_tour['ID'] 
woman_test_tour = woman_test_tour[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('woman_XGB_Phase_1_42_Optuna_Hyperparameters.csv')
xgb_preds = list()

for i in tqdm(range(100)):

    xgb_md = XGBRegressor(tree_method = 'hist', 
                          max_depth = xgb_params['max_depth'][0],
                          learning_rate = xgb_params['learning_rate'][0],
                          n_estimators = xgb_params['n_estimators'][0],
                          gamma = xgb_params['gamma'][0],
                          min_child_weight = xgb_params['min_child_weight'][0],
                          colsample_bytree = xgb_params['colsample_bytree'][0],
                          subsample = xgb_params['subsample'][0],
                          random_state = 1).fit(X, Y)

    xgb_pred = xgb_md.predict(woman_test_tour)
    xgb_preds.append(xgb_pred)
    
##############
## LightGBM ##
##############

lgb_params = pd.read_csv('woman_LightGBM_Phase_1_42_Optuna_Hyperparameters.csv')
lgb_preds = list()

for i in tqdm(range(100)):

    lgb_md = LGBMRegressor(boosting_type = 'gbdt', 
                           n_estimators = lgb_params['n_estimators'][0],
                           learning_rate = lgb_params['learning_rate'][0],
                           max_depth = lgb_params['max_depth'][0],
                           lambda_l1 = lgb_params['lambda_l1'][0],
                           lambda_l2 = lgb_params['lambda_l2'][0],
                           num_leaves = lgb_params['num_leaves'][0],
                           bagging_fraction = lgb_params['bagging_fraction'][0],
                           feature_fraction = lgb_params['feature_fraction'][0], 
                           random_state = i).fit(X, Y)

    lgb_pred = lgb_md.predict(woman_test_tour)
    lgb_preds.append(lgb_pred)
    
##################
## HistGradient ##
##################

hist_params = pd.read_csv('woman_Hist_Phase_1_42_Optuna_Hyperparameters.csv')
hist_preds = list()

for i in tqdm(range(100)):

    hist_md = HistGradientBoostingRegressor(l2_regularization = hist_params['l2_regularization'][0],
                                            early_stopping = False,
                                            learning_rate = hist_params['learning_rate'][0],
                                            max_iter = hist_params['max_iter'][0],
                                            max_depth = hist_params['max_depth'][0],
                                            max_bins = hist_params['max_bins'][0],
                                            min_samples_leaf = hist_params['min_samples_leaf'][0],
                                            max_leaf_nodes = hist_params['max_leaf_nodes'][0],
                                            random_state = i).fit(X, Y)

    hist_pred = hist_md.predict(woman_test_tour)
    hist_preds.append(hist_pred)

##############
## Ensemble ##
##############

xgb_pred = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
lgb_pred = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
hist_pred = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
woman_test_tour['ResultDiff'] = np.round_(ens_pred, decimals = 0).astype(int)
woman_test_tour['ID'] = woman_test_IDS

########################
## Appending 538 data ##
########################

data_538 = ['ID',
            'X1_team_rating',
            'X1_rd1_win',
            'X1_rd2_win',
            'X1_rd3_win',
            'X1_rd4_win',
            'X1_rd5_win',
            'X1_rd6_win',
            'X1_rd7_win',
            'X2_team_rating',
            'X2_rd1_win',
            'X2_rd2_win',
            'X2_rd3_win',
            'X2_rd4_win',
            'X2_rd5_win',
            'X2_rd6_win',
            'X2_rd7_win']

woman_test_538 = woman_test[data_538]
woman_test_tour = pd.merge(woman_test_tour, woman_test_538, on = 'ID', how = 'left')

 14%|█▍        | 14/100 [00:31<03:17,  2.29s/it]

[LightGBM] [Warning] lambda_l1 is set=9.27034195631365, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.27034195631365
[LightGBM] [Warning] bagging_fraction is set=0.2077432535958427, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2077432535958427
[LightGBM] [Warning] feature_fraction is set=0.3451001288372496, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3451001288372496
[LightGBM] [Warning] lambda_l2 is set=1.7560660843788178, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7560660843788178


 15%|█▌        | 15/100 [00:33<03:15,  2.30s/it]

[LightGBM] [Warning] lambda_l1 is set=9.27034195631365, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.27034195631365
[LightGBM] [Warning] bagging_fraction is set=0.2077432535958427, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2077432535958427
[LightGBM] [Warning] feature_fraction is set=0.3451001288372496, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3451001288372496
[LightGBM] [Warning] lambda_l2 is set=1.7560660843788178, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7560660843788178


 16%|█▌        | 16/100 [00:36<03:13,  2.30s/it]

...

 98%|█████████▊| 98/100 [03:58<00:04,  2.25s/it]

[LightGBM] [Warning] lambda_l1 is set=9.27034195631365, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.27034195631365
[LightGBM] [Warning] bagging_fraction is set=0.2077432535958427, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2077432535958427
[LightGBM] [Warning] feature_fraction is set=0.3451001288372496, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3451001288372496
[LightGBM] [Warning] lambda_l2 is set=1.7560660843788178, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7560660843788178


 99%|█████████▉| 99/100 [04:01<00:02,  2.25s/it]

[LightGBM] [Warning] lambda_l1 is set=9.27034195631365, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.27034195631365
[LightGBM] [Warning] bagging_fraction is set=0.2077432535958427, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2077432535958427
[LightGBM] [Warning] feature_fraction is set=0.3451001288372496, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3451001288372496
[LightGBM] [Warning] lambda_l2 is set=1.7560660843788178, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.7560660843788178


100%|██████████| 100/100 [04:55<00:00,  2.96s/it]


## Phase 2: Woman

In [ ]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'Seed1',
             'Seed2',
             'SeedDiff',
             'quality_march_T1',
             'quality_march_T2', 
             'ResultDiff',
             'X1_team_rating',
             'X1_rd1_win',
             'X1_rd2_win',
             'X1_rd3_win',
             'X1_rd4_win',
             'X1_rd5_win',
             'X1_rd6_win',
             'X1_rd7_win',
             'X2_team_rating',
             'X2_rd1_win',
             'X2_rd2_win',
             'X2_rd3_win',
             'X2_rd4_win',
             'X2_rd5_win',
             'X2_rd6_win',
             'X2_rd7_win']

X = woman_train_2016[to_select]
Y = woman_train_2016['target']

woman_test_tour_ID = woman_test_tour['ID']
woman_test_tour = woman_test_tour[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('woman_XGB_Phase_2_42_Optuna_Hyperparameters.csv')
xgb_preds = list()

for i in tqdm(range(100)):

    xgb_md = XGBClassifier(tree_method = 'hist', 
                           max_depth = xgb_params['max_depth'][0],
                           learning_rate = xgb_params['learning_rate'][0],
                           n_estimators = xgb_params['n_estimators'][0],
                           gamma = xgb_params['gamma'][0],
                           min_child_weight = xgb_params['min_child_weight'][0],
                           colsample_bytree = xgb_params['colsample_bytree'][0],
                           subsample = xgb_params['subsample'][0],
                           random_state = i).fit(X, Y)

    xgb_pred = xgb_md.predict_proba(woman_test_tour)[:, 1]
    xgb_preds.append(xgb_pred)

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('woman_LightGBM_Phase_2_42_Optuna_Hyperparameters.csv')
lgb_preds = list()

for i in tqdm(range(100)):
    
    lgb_md = LGBMClassifier(boosting_type = 'gbdt', 
                            n_estimators = lgb_params['n_estimators'][0],
                            learning_rate = lgb_params['learning_rate'][0],
                            max_depth = lgb_params['max_depth'][0],
                            lambda_l1 = lgb_params['lambda_l1'][0],
                            lambda_l2 = lgb_params['lambda_l2'][0],
                            num_leaves = lgb_params['num_leaves'][0],
                            bagging_fraction = lgb_params['bagging_fraction'][0],
                            feature_fraction = lgb_params['feature_fraction'][0],
                            random_state = i).fit(X, Y)

    lgb_pred = lgb_md.predict_proba(woman_test_tour)[:, 1]
    lgb_preds.append(lgb_pred)

##################
## HistGradient ##
##################

hist_params = pd.read_csv('woman_Hist_Phase_2_42_Optuna_Hyperparameters.csv')
hist_preds = list()

for i in tqdm(range(100)):

    hist_md = HistGradientBoostingClassifier(l2_regularization = hist_params['l2_regularization'][0],
                                             early_stopping = False,
                                             learning_rate = hist_params['learning_rate'][0],
                                             max_iter = hist_params['max_iter'][0],
                                             max_depth = hist_params['max_depth'][0],
                                             max_bins = hist_params['max_bins'][0],
                                             min_samples_leaf = hist_params['min_samples_leaf'][0],
                                             max_leaf_nodes = hist_params['max_leaf_nodes'][0], 
                                             random_state = i).fit(X, Y)

    hist_pred = hist_md.predict_proba(woman_test_tour)[:, 1]
    hist_preds.append(hist_pred)

##############
## Ensemble ##
##############

xgb_pred = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
lgb_pred = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
hist_pred = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
woman_pred = pd.DataFrame({'ID': woman_test_tour_ID, 'Pred': ens_pred})
woman_pred.head()

  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=0.0474587633751369, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0474587633751369
[LightGBM] [Warning] bagging_fraction is set=0.8976619661127543, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8976619661127543
[LightGBM] [Warning] feature_fraction is set=0.6539801342710443, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6539801342710443
[LightGBM] [Warning] lambda_l2 is set=0.3422853948073336, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3422853948073336


  1%|          | 1/100 [00:01<01:48,  1.10s/it]

[LightGBM] [Warning] lambda_l1 is set=0.0474587633751369, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0474587633751369
[LightGBM] [Warning] bagging_fraction is set=0.8976619661127543, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8976619661127543
[LightGBM] [Warning] feature_fraction is set=0.6539801342710443, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6539801342710443
[LightGBM] [Warning] lambda_l2 is set=0.3422853948073336, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3422853948073336


  2%|▏         | 2/100 [00:02<01:40,  1.03s/it]

...

[LightGBM] [Warning] lambda_l1 is set=0.0474587633751369, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0474587633751369
[LightGBM] [Warning] bagging_fraction is set=0.8976619661127543, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8976619661127543
[LightGBM] [Warning] feature_fraction is set=0.6539801342710443, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6539801342710443
[LightGBM] [Warning] lambda_l2 is set=0.3422853948073336, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3422853948073336


 99%|█████████▉| 99/100 [01:52<00:01,  1.13s/it]

[LightGBM] [Warning] lambda_l1 is set=0.0474587633751369, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0474587633751369
[LightGBM] [Warning] bagging_fraction is set=0.8976619661127543, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8976619661127543
[LightGBM] [Warning] feature_fraction is set=0.6539801342710443, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6539801342710443
[LightGBM] [Warning] lambda_l2 is set=0.3422853948073336, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3422853948073336


100%|██████████| 100/100 [03:51<00:00,  2.32s/it]


,ID,Pred
0,2023_3104_3112,0.093010
1,2023_3104_3124,0.081812
2,2023_3104_3151,0.903684
3,2023_3104_3156,0.899416
4,2023_3104_3160,0.078373


In [ ]:
submission = pd.merge(submission, man_pred, on = 'ID', how = 'left')
submission = pd.merge(submission, woman_pred, on = 'ID', how = 'left')
submission.columns = ['ID', 'pred1', 'pred2', 'pred3']
submission['Pred'] = np.where(np.isnan(submission['pred2']), submission['pred1'], submission['pred2'])
submission['Pred'] = np.where(np.isnan(submission['pred3']), submission['Pred'], submission['pred3'])
submission.drop(columns = ['pred1', 'pred2', 'pred3'], axis = 1, inplace = True)
submission.head()

,ID,Pred
0,2023_1101_1102,0.5
1,2023_1101_1103,0.5
2,2023_1101_1104,0.5
3,2023_1101_1105,0.5
4,2023_1101_1106,0.5


In [ ]:
submission.to_csv('Third_submission.csv', index = False)

# Solution 2
## Phase 1: Man

In [ ]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean']

X = man_train[to_select]
Y = man_train['ResultDiff']

man_test_other =  man_test[(man_test['Seed1'].isnull() | man_test['Seed2'].isnull())].reset_index(drop = True)
man_test_IDS = man_test_other['ID'] 
man_test_other = man_test_other[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('man_XGB_Phase_1_Others_42_Optuna_Hyperparameters.csv')
xgb_preds = list()

for i in tqdm(range(100)):

    xgb_md = XGBRegressor(tree_method = 'hist', 
                          max_depth = xgb_params['max_depth'][0],
                          learning_rate = xgb_params['learning_rate'][0],
                          n_estimators = xgb_params['n_estimators'][0],
                          gamma = xgb_params['gamma'][0],
                          min_child_weight = xgb_params['min_child_weight'][0],
                          colsample_bytree = xgb_params['colsample_bytree'][0],
                          subsample = xgb_params['subsample'][0],
                          random_state = 1).fit(X, Y)

    xgb_pred = xgb_md.predict(man_test_other)
    xgb_preds.append(xgb_pred)
    
##############
## LightGBM ##
##############

lgb_params = pd.read_csv('man_LightGBM_Phase_1_Others_42_Optuna_Hyperparameters.csv')
lgb_preds = list()

for i in tqdm(range(100)):

    lgb_md = LGBMRegressor(boosting_type = 'gbdt', 
                           n_estimators = lgb_params['n_estimators'][0],
                           learning_rate = lgb_params['learning_rate'][0],
                           max_depth = lgb_params['max_depth'][0],
                           lambda_l1 = lgb_params['lambda_l1'][0],
                           lambda_l2 = lgb_params['lambda_l2'][0],
                           num_leaves = lgb_params['num_leaves'][0],
                           bagging_fraction = lgb_params['bagging_fraction'][0],
                           feature_fraction = lgb_params['feature_fraction'][0], 
                           random_state = i).fit(X, Y)

    lgb_pred = lgb_md.predict(man_test_other)
    lgb_preds.append(lgb_pred)
    
##################
## HistGradient ##
##################

hist_params = pd.read_csv('man_Hist_Phase_1_Others_42_Optuna_Hyperparameters.csv')
hist_preds = list()

for i in tqdm(range(100)):

    hist_md = HistGradientBoostingRegressor(l2_regularization = hist_params['l2_regularization'][0],
                                            early_stopping = False,
                                            learning_rate = hist_params['learning_rate'][0],
                                            max_iter = hist_params['max_iter'][0],
                                            max_depth = hist_params['max_depth'][0],
                                            max_bins = hist_params['max_bins'][0],
                                            min_samples_leaf = hist_params['min_samples_leaf'][0],
                                            max_leaf_nodes = hist_params['max_leaf_nodes'][0],
                                            random_state = i).fit(X, Y)

    hist_pred = hist_md.predict(man_test_other)
    hist_preds.append(hist_pred)

##############
## Ensemble ##
##############

xgb_pred = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
lgb_pred = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
hist_pred = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
man_test_other['ResultDiff'] = np.round_(ens_pred, decimals = 0).astype(int)
man_test_other['ID'] = man_test_IDS

  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=0.0768920138186167, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0768920138186167
[LightGBM] [Warning] bagging_fraction is set=0.576781303889645, subsample=1.0 will be ignored. Current value: bagging_fraction=0.576781303889645
[LightGBM] [Warning] feature_fraction is set=0.548497029189781, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.548497029189781
[LightGBM] [Warning] lambda_l2 is set=0.0595126440974096, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0595126440974096


  1%|          | 1/100 [00:17<29:19, 17.77s/it]

[LightGBM] [Warning] lambda_l1 is set=0.0768920138186167, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0768920138186167
[LightGBM] [Warning] bagging_fraction is set=0.576781303889645, subsample=1.0 will be ignored. Current value: bagging_fraction=0.576781303889645
[LightGBM] [Warning] feature_fraction is set=0.548497029189781, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.548497029189781
[LightGBM] [Warning] lambda_l2 is set=0.0595126440974096, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0595126440974096


  2%|▏         | 2/100 [00:25<19:29, 11.93s/it]

...

 98%|█████████▊| 98/100 [14:14<00:16,  8.24s/it]

[LightGBM] [Warning] lambda_l1 is set=0.0768920138186167, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0768920138186167
[LightGBM] [Warning] bagging_fraction is set=0.576781303889645, subsample=1.0 will be ignored. Current value: bagging_fraction=0.576781303889645
[LightGBM] [Warning] feature_fraction is set=0.548497029189781, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.548497029189781
[LightGBM] [Warning] lambda_l2 is set=0.0595126440974096, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0595126440974096


 99%|█████████▉| 99/100 [14:22<00:08,  8.24s/it]

[LightGBM] [Warning] lambda_l1 is set=0.0768920138186167, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0768920138186167
[LightGBM] [Warning] bagging_fraction is set=0.576781303889645, subsample=1.0 will be ignored. Current value: bagging_fraction=0.576781303889645
[LightGBM] [Warning] feature_fraction is set=0.548497029189781, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.548497029189781
[LightGBM] [Warning] lambda_l2 is set=0.0595126440974096, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0595126440974096


100%|██████████| 100/100 [26:15<00:00, 15.76s/it]


## Phase 2: Man

In [ ]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'ResultDiff']

X = man_train[to_select]
Y = man_train['target']

man_test_other_ID = man_test_other['ID']
man_test_other = man_test_other[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('man_XGB_Phase_2_Others_42_Optuna_Hyperparameters.csv')
xgb_preds = list()

for i in tqdm(range(100)):

    xgb_md = XGBClassifier(tree_method = 'hist', 
                           max_depth = xgb_params['max_depth'][0],
                           learning_rate = xgb_params['learning_rate'][0],
                           n_estimators = xgb_params['n_estimators'][0],
                           gamma = xgb_params['gamma'][0],
                           min_child_weight = xgb_params['min_child_weight'][0],
                           colsample_bytree = xgb_params['colsample_bytree'][0],
                           subsample = xgb_params['subsample'][0],
                           random_state = i).fit(X, Y)

    xgb_pred = xgb_md.predict_proba(man_test_other)[:, 1]
    xgb_preds.append(xgb_pred)

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('man_LightGBM_Phase_2_Others_42_Optuna_Hyperparameters.csv')
lgb_preds = list()

for i in tqdm(range(50)):
    
    lgb_md = LGBMClassifier(boosting_type = 'gbdt', 
                            n_estimators = lgb_params['n_estimators'][0],
                            learning_rate = lgb_params['learning_rate'][0],
                            max_depth = lgb_params['max_depth'][0],
                            lambda_l1 = lgb_params['lambda_l1'][0],
                            lambda_l2 = lgb_params['lambda_l2'][0],
                            num_leaves = lgb_params['num_leaves'][0],
                            bagging_fraction = lgb_params['bagging_fraction'][0],
                            feature_fraction = lgb_params['feature_fraction'][0],
                            random_state = i).fit(X, Y)

    lgb_pred = lgb_md.predict_proba(man_test_other)[:, 1]
    lgb_preds.append(lgb_pred)

##################
## HistGradient ##
##################

hist_params = pd.read_csv('man_Hist_Phase_2_Others_42_Optuna_Hyperparameters.csv')
hist_preds = list()

for i in tqdm(range(50)):

    hist_md = HistGradientBoostingClassifier(l2_regularization = hist_params['l2_regularization'][0],
                                             early_stopping = False,
                                             learning_rate = hist_params['learning_rate'][0],
                                             max_iter = hist_params['max_iter'][0],
                                             max_depth = hist_params['max_depth'][0],
                                             max_bins = hist_params['max_bins'][0],
                                             min_samples_leaf = hist_params['min_samples_leaf'][0],
                                             max_leaf_nodes = hist_params['max_leaf_nodes'][0], 
                                             random_state = i).fit(X, Y)

    hist_pred = hist_md.predict_proba(man_test_other)[:, 1]
    hist_preds.append(hist_pred)

##############
## Ensemble ##
##############

xgb_pred = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
lgb_pred = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
hist_pred = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
man_pred_other = pd.DataFrame({'ID': man_test_other_ID, 'Pred': ens_pred})
man_pred_other.head()

  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=0.0101037077177569, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0101037077177569
[LightGBM] [Warning] bagging_fraction is set=0.8808570379580776, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8808570379580776
[LightGBM] [Warning] feature_fraction is set=0.8912915369733756, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8912915369733756
[LightGBM] [Warning] lambda_l2 is set=0.0104110642739638, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0104110642739638


  1%|          | 1/100 [00:02<03:56,  2.39s/it]

[LightGBM] [Warning] lambda_l1 is set=0.0101037077177569, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0101037077177569
[LightGBM] [Warning] bagging_fraction is set=0.8808570379580776, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8808570379580776
[LightGBM] [Warning] feature_fraction is set=0.8912915369733756, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8912915369733756
[LightGBM] [Warning] lambda_l2 is set=0.0104110642739638, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0104110642739638


  2%|▏         | 2/100 [00:04<03:37,  2.22s/it]

...

[LightGBM] [Warning] lambda_l1 is set=0.0101037077177569, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0101037077177569
[LightGBM] [Warning] bagging_fraction is set=0.8808570379580776, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8808570379580776
[LightGBM] [Warning] feature_fraction is set=0.8912915369733756, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8912915369733756
[LightGBM] [Warning] lambda_l2 is set=0.0104110642739638, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0104110642739638


 99%|█████████▉| 99/100 [03:46<00:02,  2.33s/it]

[LightGBM] [Warning] lambda_l1 is set=0.0101037077177569, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0101037077177569
[LightGBM] [Warning] bagging_fraction is set=0.8808570379580776, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8808570379580776
[LightGBM] [Warning] feature_fraction is set=0.8912915369733756, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8912915369733756
[LightGBM] [Warning] lambda_l2 is set=0.0104110642739638, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0104110642739638


100%|██████████| 100/100 [05:40<00:00,  3.40s/it]


,ID,Pred
0,2023_1101_1102,0.000814
1,2023_1101_1103,0.000730
2,2023_1101_1104,0.000457
3,2023_1101_1105,0.000853
4,2023_1101_1106,0.999231


## Phase 1: Woman

In [12]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean']

X = woman_train[to_select]
Y = woman_train['ResultDiff']

woman_test_other =  woman_test[(woman_test['Seed1'].isnull() | woman_test['Seed2'].isnull())].reset_index(drop = True)
woman_test_IDS = woman_test_other['ID'] 
woman_test_other = woman_test_other[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('woman_XGB_Phase_1_Others_42_Optuna_Hyperparameters.csv')
xgb_preds = list()

for i in tqdm(range(30)):

    xgb_md = XGBRegressor(tree_method = 'hist', 
                          max_depth = xgb_params['max_depth'][0],
                          learning_rate = xgb_params['learning_rate'][0],
                          n_estimators = xgb_params['n_estimators'][0],
                          gamma = xgb_params['gamma'][0],
                          min_child_weight = xgb_params['min_child_weight'][0],
                          colsample_bytree = xgb_params['colsample_bytree'][0],
                          subsample = xgb_params['subsample'][0],
                          random_state = 1).fit(X, Y)

    xgb_pred = xgb_md.predict(woman_test_other)
    xgb_preds.append(xgb_pred)
    
##############
## LightGBM ##
##############

lgb_params = pd.read_csv('woman_LightGBM_Phase_1_Others_42_Optuna_Hyperparameters.csv')
lgb_preds = list()

for i in tqdm(range(30)):

    lgb_md = LGBMRegressor(boosting_type = 'gbdt', 
                           n_estimators = lgb_params['n_estimators'][0],
                           learning_rate = lgb_params['learning_rate'][0],
                           max_depth = lgb_params['max_depth'][0],
                           lambda_l1 = lgb_params['lambda_l1'][0],
                           lambda_l2 = lgb_params['lambda_l2'][0],
                           num_leaves = lgb_params['num_leaves'][0],
                           bagging_fraction = lgb_params['bagging_fraction'][0],
                           feature_fraction = lgb_params['feature_fraction'][0], 
                           random_state = i).fit(X, Y)

    lgb_pred = lgb_md.predict(woman_test_other)
    lgb_preds.append(lgb_pred)
    
##################
## HistGradient ##
##################

hist_params = pd.read_csv('woman_Hist_Phase_1_Others_42_Optuna_Hyperparameters.csv')
hist_preds = list()

for i in tqdm(range(30)):

    hist_md = HistGradientBoostingRegressor(l2_regularization = hist_params['l2_regularization'][0],
                                            early_stopping = False,
                                            learning_rate = hist_params['learning_rate'][0],
                                            max_iter = hist_params['max_iter'][0],
                                            max_depth = hist_params['max_depth'][0],
                                            max_bins = hist_params['max_bins'][0],
                                            min_samples_leaf = hist_params['min_samples_leaf'][0],
                                            max_leaf_nodes = hist_params['max_leaf_nodes'][0],
                                            random_state = i).fit(X, Y)

    hist_pred = hist_md.predict(woman_test_other)
    hist_preds.append(hist_pred)

##############
## Ensemble ##
##############

xgb_pred = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
lgb_pred = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
hist_pred = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
woman_test_other['ResultDiff'] = np.round_(ens_pred, decimals = 0).astype(int)
woman_test_other['ID'] = woman_test_IDS

  0%|          | 0/30 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=4.655160825904243, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.655160825904243
[LightGBM] [Warning] bagging_fraction is set=0.6340177598805505, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6340177598805505
[LightGBM] [Warning] feature_fraction is set=0.6056599038672632, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6056599038672632
[LightGBM] [Warning] lambda_l2 is set=0.0338137894715804, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0338137894715804


  3%|▎         | 1/30 [00:09<04:28,  9.24s/it]

[LightGBM] [Warning] lambda_l1 is set=4.655160825904243, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.655160825904243
[LightGBM] [Warning] bagging_fraction is set=0.6340177598805505, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6340177598805505
[LightGBM] [Warning] feature_fraction is set=0.6056599038672632, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6056599038672632
[LightGBM] [Warning] lambda_l2 is set=0.0338137894715804, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0338137894715804


  7%|▋         | 2/30 [00:24<05:56, 12.73s/it]

...

 93%|█████████▎| 28/30 [04:09<00:17,  8.82s/it]

[LightGBM] [Warning] lambda_l1 is set=4.655160825904243, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.655160825904243
[LightGBM] [Warning] bagging_fraction is set=0.6340177598805505, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6340177598805505
[LightGBM] [Warning] feature_fraction is set=0.6056599038672632, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6056599038672632
[LightGBM] [Warning] lambda_l2 is set=0.0338137894715804, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0338137894715804


 97%|█████████▋| 29/30 [04:17<00:08,  8.76s/it]

[LightGBM] [Warning] lambda_l1 is set=4.655160825904243, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.655160825904243
[LightGBM] [Warning] bagging_fraction is set=0.6340177598805505, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6340177598805505
[LightGBM] [Warning] feature_fraction is set=0.6056599038672632, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6056599038672632
[LightGBM] [Warning] lambda_l2 is set=0.0338137894715804, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0338137894715804


100%|██████████| 30/30 [03:38<00:00,  7.28s/it]


## Phase 2: Woman

In [13]:
to_select = ['X1_WinRatio14d',
             'X1_PointsMean',
             'X1_PointsMedian',
             'X1_PointsDiffMean',
             'X1_FgaMean',
             'X1_FgaMedian',
             'X1_FgaMin',
             'X1_FgaMax',
             'X1_AstMean',
             'X1_BlkMean',
             'X1_OppFgaMean',
             'X1_OppFgaMin',
             'X1_EfgpMean',
             'X1_PossessionsMean',
             'X1_PpmMean',
             'X1_FtrMean',
             'X1_TopMean',
             'X1_DrebpMean',
             'X2_WinRatio14d',
             'X2_PointsMean',
             'X2_PointsMedian',
             'X2_PointsDiffMean',
             'X2_FgaMean',
             'X2_FgaMedian',
             'X2_FgaMin',
             'X2_FgaMax',
             'X2_AstMean',
             'X2_BlkMean',
             'X2_OppFgaMean',
             'X2_OppFgaMin',
             'X2_EfgpMean',
             'X2_PossessionsMean',
             'X2_PpmMean',
             'X2_FtrMean',
             'X2_TopMean',
             'X2_DrebpMean',
             'ResultDiff']

X = woman_train[to_select]
Y = woman_train['target']

woman_test_other_ID = woman_test_other['ID']
woman_test_other = woman_test_other[to_select]

#############
## XGBoost ##
#############

xgb_params = pd.read_csv('woman_XGB_Phase_2_Others_42_Optuna_Hyperparameters.csv')
xgb_preds = list()

for i in tqdm(range(20)):

    xgb_md = XGBClassifier(tree_method = 'hist', 
                           max_depth = xgb_params['max_depth'][0],
                           learning_rate = xgb_params['learning_rate'][0],
                           n_estimators = xgb_params['n_estimators'][0],
                           gamma = xgb_params['gamma'][0],
                           min_child_weight = xgb_params['min_child_weight'][0],
                           colsample_bytree = xgb_params['colsample_bytree'][0],
                           subsample = xgb_params['subsample'][0],
                           random_state = i).fit(X, Y)

    xgb_pred = xgb_md.predict_proba(woman_test_other)[:, 1]
    xgb_preds.append(xgb_pred)

##############
## LightGBM ##
##############

lgb_params = pd.read_csv('woman_LightGBM_Phase_2_Others_42_Optuna_Hyperparameters.csv')
lgb_preds = list()

for i in tqdm(range(20)):
    
    lgb_md = LGBMClassifier(boosting_type = 'gbdt', 
                            n_estimators = lgb_params['n_estimators'][0],
                            learning_rate = lgb_params['learning_rate'][0],
                            max_depth = lgb_params['max_depth'][0],
                            lambda_l1 = lgb_params['lambda_l1'][0],
                            lambda_l2 = lgb_params['lambda_l2'][0],
                            num_leaves = lgb_params['num_leaves'][0],
                            bagging_fraction = lgb_params['bagging_fraction'][0],
                            feature_fraction = lgb_params['feature_fraction'][0],
                            random_state = i).fit(X, Y)

    lgb_pred = lgb_md.predict_proba(woman_test_other)[:, 1]
    lgb_preds.append(lgb_pred)

##################
## HistGradient ##
##################

hist_params = pd.read_csv('woman_Hist_Phase_2_Others_42_Optuna_Hyperparameters.csv')
hist_preds = list()

for i in tqdm(range(20)):

    hist_md = HistGradientBoostingClassifier(l2_regularization = hist_params['l2_regularization'][0],
                                             early_stopping = False,
                                             learning_rate = hist_params['learning_rate'][0],
                                             max_iter = hist_params['max_iter'][0],
                                             max_depth = hist_params['max_depth'][0],
                                             max_bins = hist_params['max_bins'][0],
                                             min_samples_leaf = hist_params['min_samples_leaf'][0],
                                             max_leaf_nodes = hist_params['max_leaf_nodes'][0], 
                                             random_state = i).fit(X, Y)

    hist_pred = hist_md.predict_proba(woman_test_other)[:, 1]
    hist_preds.append(hist_pred)

##############
## Ensemble ##
##############

xgb_pred = pd.DataFrame(xgb_preds).apply(np.mean, axis = 0)
lgb_pred = pd.DataFrame(lgb_preds).apply(np.mean, axis = 0)
hist_pred = pd.DataFrame(hist_preds).apply(np.mean, axis = 0)

ens_pred = (xgb_pred + lgb_pred + hist_pred) / 3
woman_pred_other = pd.DataFrame({'ID': woman_test_other_ID, 'Pred': ens_pred})
woman_pred_other.head()

  0%|          | 0/20 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=0.0135218970531824, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0135218970531824
[LightGBM] [Warning] bagging_fraction is set=0.5621118842421864, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5621118842421864
[LightGBM] [Warning] feature_fraction is set=0.4297732298939334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4297732298939334
[LightGBM] [Warning] lambda_l2 is set=1.35554745456749, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.35554745456749


  5%|▌         | 1/20 [00:00<00:12,  1.52it/s]

[LightGBM] [Warning] lambda_l1 is set=0.0135218970531824, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0135218970531824
[LightGBM] [Warning] bagging_fraction is set=0.5621118842421864, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5621118842421864
[LightGBM] [Warning] feature_fraction is set=0.4297732298939334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4297732298939334
[LightGBM] [Warning] lambda_l2 is set=1.35554745456749, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.35554745456749


 10%|█         | 2/20 [00:01<00:12,  1.50it/s]

...

[LightGBM] [Warning] lambda_l1 is set=0.0135218970531824, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0135218970531824
[LightGBM] [Warning] bagging_fraction is set=0.5621118842421864, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5621118842421864
[LightGBM] [Warning] feature_fraction is set=0.4297732298939334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4297732298939334
[LightGBM] [Warning] lambda_l2 is set=1.35554745456749, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.35554745456749


 95%|█████████▌| 19/20 [00:12<00:00,  1.58it/s]

[LightGBM] [Warning] lambda_l1 is set=0.0135218970531824, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0135218970531824
[LightGBM] [Warning] bagging_fraction is set=0.5621118842421864, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5621118842421864
[LightGBM] [Warning] feature_fraction is set=0.4297732298939334, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4297732298939334
[LightGBM] [Warning] lambda_l2 is set=1.35554745456749, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.35554745456749


100%|██████████| 20/20 [00:18<00:00,  1.10it/s]


,ID,Pred
0,2023_3101_3102,0.997251
1,2023_3101_3103,0.997155
2,2023_3101_3104,0.002651
3,2023_3101_3105,0.997242
4,2023_3101_3106,0.997230


In [14]:
submission = pd.merge(submission, man_pred_other, on = 'ID', how = 'left')
submission = pd.merge(submission, woman_pred_other, on = 'ID', how = 'left')
submission.columns = ['ID', 'pred1', 'pred2', 'pred3']
submission['Pred'] = np.where(np.isnan(submission['pred2']), submission['pred1'], submission['pred2'])
submission['Pred'] = np.where(np.isnan(submission['pred3']), submission['Pred'], submission['pred3'])
submission.drop(columns = ['pred1', 'pred2', 'pred3'], axis = 1, inplace = True)
submission.head()

,ID,Pred
0,2023_1101_1102,0.000814
1,2023_1101_1103,0.000730
2,2023_1101_1104,0.000457
3,2023_1101_1105,0.000853
4,2023_1101_1106,0.999231


In [15]:
submission.to_csv('Fourth_submission.csv', index = False)